# Color Skymaps for GW Image Paper
Ira Thorpe, 2022.11.12
Based on the work of Kaitlyn Szerkezces in Summer 2022


### Setup

In [ ]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import FormatStrFormatter
import sys
from lisacattools import HPhist, convert_ecliptic_to_galactic

# import GW Imager libraries
sys.path.append('../src/')
import constants
import metrics
import sources
import concepts
import subsystems
import background
import falseColorGW

# make directory for plots
!mkdir -p '../plots/'

fontSize = 20


In [ ]:
# Define mission concept
model = concepts.LISASciRDv1
#model = concepts.AMIGO

model = background.add2model(model)
model['label']= 'LISA'
model

In [ ]:
# Load in the GBs from LDC
LDCgalaxyFile = **FILENAME**
galaxy = pd.read_hdf(LDCgalaxyFile,key='keyfile')
galaxy.describe()

In [ ]:
# Get the SNR and CDL for this galaxy with this detector
f = galaxy['Frequency']
h = galaxy['Amplitude']
snr = metrics.getCWsnr(f,h,4*constants.year,model)
lamGW = constants.c / f
B = 2.0*constants.AU*constants.c
cdl = (1/snr)*(lamGW/B)*180/np.pi

galaxy.insert(8,'SNR',snr)
galaxy.insert(9,'CDL', cdl)
galaxy.describe()

In [ ]:
# make an SNR cut to estimate detections
galaxy_detected = galaxy[galaxy['SNR']>=8]
galaxy_detected.describe()

In [ ]:
# Frequency/Amplitude scatter plot for detections, with colorbars showing CDL
fig = plt.figure(figsize=(10, 10), dpi = 100)
ax = plt.axes()

plt.yscale('log')
plt.xscale('log')
plt.xlim([3e-4,3e-2])
plt.ylim([1e-25,1e-20])
ax.tick_params(axis = 'both', which = 'major', labelsize = 12)

#cNorm = colors.LogNorm(vmin=galaxy_detected['CDL'].min(), vmax=galaxy_detected['CDL'].max()) #re-wrapping normalization
cNorm = colors.LogNorm(vmin=3e-3, vmax=3e1) #re-wrapping normalization

scalarMap = cm.ScalarMappable(norm=cNorm, cmap=plt.cm.get_cmap('plasma_r'))

galaxy_detected.plot(
    kind='scatter', 
    x='Frequency', 
    y='Amplitude',  
    marker = '.',
    s = 2,
    c = scalarMap.to_rgba(np.array(galaxy_detected['CDL'])),
    ax = ax);

ax.grid()
plt.xlabel('Frequency [Hz]',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
plt.ylabel('Strain Amplitude',fontsize=fontSize)
plt.yticks(fontsize=fontSize)

f = np.logspace(-4,0,512)
inst = metrics.makeSensitivity(f,model)
ax.plot(f,np.sqrt(inst/(4*constants.year)), color='k',linestyle = '--')

ax.legend(['%s Sensitivity' % model['label'],'Detected UCBs'],fontsize=fontSize)

cbar = fig.colorbar(scalarMap)
cbar.set_label('CDL [deg]',fontsize=fontSize)
cbar.ax.tick_params(labelsize=fontSize)
fig.savefig('../plots/%s_cdl.png' % model['label'])

## Make the all-sky Image

In [ ]:
# convert the sources to galactic coordinates and rotate to place galactic center at longitude = 0 with extents -180,180 
convert_ecliptic_to_galactic(galaxy_detected)
all_sources_left = galaxy_detected[galaxy_detected['Galactic Longitude']<=180].copy()
all_sources_right = galaxy_detected[galaxy_detected['Galactic Longitude']>180].copy()
lon_rot = np.array(all_sources_right['Galactic Longitude'])-360
all_sources_right.loc[:,'Galactic Longitude'] = lon_rot
all_sources = pd.concat((all_sources_right,all_sources_left))
all_sources.hist(column='Galactic Longitude',bins=100)

In [ ]:
# make a super ensemble
super_ensemble = falseColorGW.makeSuperEnsemble(all_sources,cdlColName = 'CDL')
super_ensemble.describe()

In [ ]:
# make the spectrum as number of sources
fmax = 12e-3
cmap = plt.get_cmap('gist_rainbow')
super_ensemble,f_bins,fmap = falseColorGW.makeGWluminosity(super_ensemble,fmax=fmax)

fig, ax = plt.subplots(1,1,figsize=(10,8),dpi=100)
def scale_x(tick_val, tick_pos):
    tmp = tick_val*1e3
    return('%1.0f' % tmp)

def scale_y(tick_val, tick_pos):
    tmp = tick_val/1e3
    return('%2.0f' % tmp)

ax.xaxis.set_major_formatter(scale_x)
ax.yaxis.set_major_formatter(scale_y)

binvals, bins, patches = ax.hist(np.array(super_ensemble['Frequency']),bins=f_bins)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

for x, p in zip(bin_centers, patches):
    p.set_facecolor(cmap(x/fmax))
ax.grid(True)

ax.set_xlim(0,fmax)
ax.set_xlabel('Frequency [mHz]',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
ax.set_ylabel('kCounts',fontsize=fontSize)
plt.yticks(fontsize=fontSize)

fig.savefig('../plots/%s_counts.png' % model['label'])



    

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8),dpi=100)

binvals, bins, patches = ax.hist(np.array(super_ensemble['Frequency']),bins=f_bins,weights=super_ensemble['Power'])

def scale_x(tick_val,tick_pos):
    tmp = tick_val*1e3
    return('%1.0f' % tmp)

def scale_y(tick_val,tick_pos):
    tmp = tick_val/np.max(binvals)
    return('%1.1f' % tmp)

ax.xaxis.set_major_formatter(scale_x)
ax.yaxis.set_major_formatter(scale_y)

bin_centers = 0.5 * (bins[:-1] + bins[1:])    

for x, p in zip(bin_centers, patches):
    p.set_facecolor(cmap(x/fmax))
ax.grid(True)
ax.set_xlim(0,fmax)
ax.set_xlabel('Frequency [mHz]',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
ax.set_ylim(0,np.max(binvals))
ax.set_ylabel('Relative Intensity',fontsize=fontSize)
plt.yticks(fontsize=fontSize)

fig.savefig('../plots/%s_intensity.png' % model['label'])

In [ ]:
# Make the image
gwimage = falseColorGW.makeGWimage(super_ensemble,fmap, skyBins = [512,1024])

In [ ]:
# Plot the Image
fig, ax = plt.subplots(1,1,figsize=(16,10),dpi=200)
ax.imshow(gwimage,extent=[-180,180,-90,90])
ax.set_ylabel('Galactic Latitude [deg]',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
ax.set_xlabel('Galactic Longitude [deg]',fontsize=fontSize)
plt.yticks(fontsize=fontSize)
fig.savefig('../plots/%s_all_sky.png' % model['label'])



In [ ]:
# plot the RGB histogram
fig, ax = plt.subplots(1,1,figsize=(10,8))
for idx, color, alpha  in zip(range(0,3), ['red','green','blue'],np.linspace(1,0.3,3)) :
    ftmp = gwimage[:,:,idx].ravel()
    ax.hist((ftmp[ftmp > 0]),color=color,log=True,bins=100,alpha=alpha)

ax.grid(True)
ax.set_ylabel('Log Counts',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
ax.set_xlabel('Log Intensity',fontsize=fontSize)
plt.yticks(fontsize=fontSize)
fig.savefig('../plots/%s_RGB_histogram.png' % model['label'])


### Make a Zoom-in Image

In [ ]:
plotRange = [-10,10,-10,10]

zoom_super_ensemble = super_ensemble[(super_ensemble['Galactic Longitude']>=plotRange[0]) & (super_ensemble['Galactic Longitude']<=plotRange[1]) & (super_ensemble['Galactic Latitude']>=plotRange[2]) & (super_ensemble['Galactic Latitude']<=plotRange[3])]
zoom_super_ensemble.describe()

In [ ]:
# Make the image
zoom_super_ensemble,f_bins,fmap = falseColorGW.makeGWluminosity(zoom_super_ensemble,fmax=fmax)
zoom_gwimage = falseColorGW.makeGWimage(zoom_super_ensemble,fmap, skyRange = plotRange, skyBins = [512,512])

In [ ]:
# Plot the Image
fig, ax = plt.subplots(1,1,figsize=(12,12))
ax.imshow(zoom_gwimage,extent = plotRange)
ax.set_ylabel('Galactic Latitude [deg]',fontsize=fontSize)
plt.xticks(fontsize=fontSize)
ax.set_xlabel('Galactic Longitude [deg]',fontsize=fontSize)
plt.yticks(fontsize=fontSize)
fig.savefig('../plots/%s_zoom.png' % model['label']) 